In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import codecs


In [2]:
fileObj = codecs.open( "data.txt", "r", "utf_8_sig" )
data= fileObj.read()
data=data.replace("\n",'')
data=data.replace("\r",'')
fileObj.close()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 116959 total characters and 83 unique characters in your data.


In [21]:
data[:2000]

'Бұл жасқа келгенше жақсы өткіздік пе, жаман өткіздік пе, әйтеуір бірталай өмірімізді өткіздік: алыстық, жұлыстық, айтыстық, тартыстық - әурешілікті көре-көре келдік. Енді жер ортасы жасқа келдік: қажыдық, жалықтық; қылып жүрген ісіміздің баянсызын, байлаусызын көрдік, бәрі қоршылық екенін білдік. Ал, енді қалған өмірімізді қайтіп, не қылып өткіземіз? Соны таба алмай өзім де қайранмын.Ел бағу? Жоқ, елге бағым жоқ. Бағусыз дертке ұшырайын деген кісі бақпаса, не албыртқан, көңілі басылмаған жастар бағамын демесе, бізді құдай сақтасын!Мал бағу? Жоқ, баға алмаймын. Балалар өздеріне керегінше өздері бағар. Енді қартайғанда қызығын өзің түгел көре алмайтұғын, ұры, залым, тілемсектердің азығын бағып беремін деп, қалған аз ғана өмірімді қор қылар жайым жоқ.Ғылым бағу? Жоқ, ғылым бағарға да ғылым сөзін сөйлесер адам жоқ. Білгеніңді кімге үйретерсің, білмегеніңді кімнен сұрарсың? Елсіз-күнсізде кездемені жайып салып, қолына кезін алып отырғанның не пайдасы бар? Мұңдасып шер тарқатысар кісі болма

In [3]:
# unique characters
char_to_ix = { ch:i for i,ch in (enumerate(sorted(chars))) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)
encoded = np.array([char_to_ix[ch] for ch in data])

{0: ' ', 1: '!', 2: '*', 3: ',', 4: '-', 5: '.', 6: ':', 7: ';', 8: '?', 9: '«', 10: '»', 11: 'І', 12: 'А', 13: 'Б', 14: 'Г', 15: 'Д', 16: 'Е', 17: 'Ж', 18: 'З', 19: 'И', 20: 'К', 21: 'Л', 22: 'М', 23: 'Н', 24: 'О', 25: 'П', 26: 'Р', 27: 'С', 28: 'Т', 29: 'У', 30: 'Ф', 31: 'Х', 32: 'Ш', 33: 'Ы', 34: 'Я', 35: 'а', 36: 'б', 37: 'в', 38: 'г', 39: 'д', 40: 'е', 41: 'ж', 42: 'з', 43: 'и', 44: 'й', 45: 'к', 46: 'л', 47: 'м', 48: 'н', 49: 'о', 50: 'п', 51: 'р', 52: 'с', 53: 'т', 54: 'у', 55: 'ф', 56: 'х', 57: 'ц', 58: 'ч', 59: 'ш', 60: 'ы', 61: 'ь', 62: 'э', 63: 'ю', 64: 'я', 65: 'і', 66: 'Ғ', 67: 'ғ', 68: 'Қ', 69: 'қ', 70: 'ң', 71: 'Ү', 72: 'ү', 73: 'Ұ', 74: 'ұ', 75: 'Һ', 76: 'һ', 77: 'Ә', 78: 'ә', 79: 'Ө', 80: 'ө', 81: '–', 82: '—'}


In [4]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')

Training on GPU!


In [5]:
def one_hot(arr, n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot

In [6]:
def get_batches(arr, batch_size, seq_length):
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size, -1))
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [7]:
class RNN(nn.Module):
    def __init__ (self, chars, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.chars = chars
        self.char_to_ix = { ch:i for i,ch in enumerate(sorted(self.chars)) }
        self.ix_to_char = { i:ch for i,ch in enumerate(sorted(self.chars)) }
        #set lstm
        self.lstm = nn.LSTM(len(self.chars),n_hidden,n_layers,dropout = drop_prob, batch_first = True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.chars))
    def forward(self, x, hidden):
        rel, hidden = self.lstm(x, hidden)
        out = self.dropout(rel)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        
        return hidden
         

In [8]:
def train(rnn, data, epochs = 10, lr = 0.001, batch_size = 10 ,seq_length = 50, clip=5, val_frac=0.1, print_every=10):
    rnn.train()
    optimization = torch.optim.Adam(rnn.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    data, val_data = data[:int(len(data)*(1-val_frac))], data[int(len(data)*(1-val_frac)):]
    if(train_on_gpu):
        rnn.cuda()
    counter = 0
    n_chars = len(rnn.chars)
  
    for e in range(epochs):
        n_h = rnn.init_hidden(batch_size)
    
        for x,y in get_batches(data, batch_size, seq_length):
            counter = counter + 1
            x = one_hot(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
            n_h = tuple([every.data for every in n_h])
            rnn.zero_grad()
            out, n_h = rnn(inputs, n_h)
            loss = criterion(out, targets.view(batch_size*seq_length).long())
            loss.backward()
            nn.utils.clip_grad_norm_(rnn.parameters(), clip)
            optimization.step()
            if counter % print_every == 0:
                val_h = rnn.init_hidden(batch_size)
                val_losses = []
                rnn.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    val_h = tuple([every.data for every in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    out, val_h = rnn(inputs, val_h)
                    val_loss = criterion(out, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                rnn.train()
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()))
  
  

In [9]:
n_hidden=512
n_layers=2

rnn = RNN(chars, n_hidden, n_layers)
print(rnn)
batch_size = 128
seq_length = 100
n_epochs = 250
train(rnn, encoded, epochs=n_epochs,lr=0.001, batch_size=batch_size, seq_length=seq_length, print_every=50)

RNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)
Epoch: 7/250... Step: 50... Loss: 3.2710...
Epoch: 13/250... Step: 100... Loss: 3.1295...
Epoch: 19/250... Step: 150... Loss: 2.4925...
Epoch: 25/250... Step: 200... Loss: 2.3396...
Epoch: 32/250... Step: 250... Loss: 2.1780...
Epoch: 38/250... Step: 300... Loss: 2.0822...
Epoch: 44/250... Step: 350... Loss: 2.0203...
Epoch: 50/250... Step: 400... Loss: 1.9764...
Epoch: 57/250... Step: 450... Loss: 1.8786...
Epoch: 63/250... Step: 500... Loss: 1.7927...
Epoch: 69/250... Step: 550... Loss: 1.7626...
Epoch: 75/250... Step: 600... Loss: 1.7381...
Epoch: 82/250... Step: 650... Loss: 1.6441...
Epoch: 88/250... Step: 700... Loss: 1.5676...
Epoch: 94/250... Step: 750... Loss: 1.5453...
Epoch: 100/250... Step: 800... Loss: 1.4976...
Epoch: 107/250... Step: 850... Loss: 1.4053...
Epoch: 113/250... Step: 900... Loss: 1.

In [10]:
def predict(rnn, char, n_h=None, top_k=None):
       
        x = np.array([[rnn.char_to_ix[char]]])
        x = one_hot(x, len(rnn.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
   
        n_h = tuple([every.data for every in n_h])
       
        out, n_h = rnn(inputs, n_h)

       
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() 
        
    
        if top_k is None:
            top_ch = np.arange(len(rnn.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        
        return rnn.ix_to_char[char], n_h

In [11]:
def sample(rnn, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        rnn.cuda()
    else:
        rnn.cpu()
    
    rnn.eval() 
    
   
    chars = [ch for ch in prime]
    n_h = rnn.init_hidden(1)
    for ch in prime:
        char, n_h = predict(rnn, ch, n_h, top_k=top_k)

    chars.append(char)
    
    
    for ii in range(size):
        char, n_h = predict(rnn, chars[-1], n_h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [12]:
print(sample(rnn, 1000, prime='А', top_k=5))

Ақылсаң берік елгенсе, осындай жіберген жоқ дой, оным жүнге  біріп ұғыным. Және манданың өзінде біргені бар болса, ауылы жаратпан, өнгерінің көңілінде ақыл қылып, өзі де уайымызға жақатына саламақ еді, өні ді үсін шайға барынды жалып, сізінің бағыпы, жөті мастынеп, артылары бар болса, айтылған себеп балалық шалыққандары алда, жаратқан жоқ деп анамендей біреуіне салып болады. Бұл ғадаләт рафы болмасыңдар. Олай болғанда қиыр жұратып жүрмес пе? Олар болсанда қайметемен түрлі - сұларға тұр болсын. Оның жәбебі нарлифа бұр ғадаләттен шығады. Оның үшін мәхкірленене себеп, наһәрларыманан сауға алады, хасуа да харат болмаса, не қылған мендеге ештіренен айтқан болса, бұрат болмақсызын бәр бар, зызын өзімен қарғана кісі бермей болады. Оның жарғы барып, жауыны танып, адам балысына пейделіктен көргенде, біргеніңде болып қалып, мақтайды қартайлық қайырлықып, жұрып болар, мақтаншақ мен ойлапайды.Дүнлеге қуаныш бар қолған кісінің, қосындан өткізіп, мешілім-жерге дүгиетің, жоқсы, ана екеуін жымдылық, с

In [13]:
print(sample(rnn, 1000, prime='Б', top_k=5))

Бісен деп асартар емес, соғим барлықты камдым қайдап қара жасқа қорақтат да жардасын деп қайтып, кеңерін болады. Жән бір өздері жарым адамдығынан жақты бер! Ол адамның өнірімеғі еңді болсын, әрдісінің өзінің адамдығынаны табын болады. Әуелі — ай, нәрсенің ретінің сөзге үйренеп екен. Қазақ денберек хайықта жұқ болса, өнер құдай тағала бір қалмаған құдійері. Мантаның шарық қылған құдай тағала айғып кетеді.Екінші - өзіңдің өскізін сынатұғын аламын да болған. Бұлар тынымандың ішкі сыларып саласып келсе, аланды баса жазалардан орлық бар. Бұлар да аудан саңпасың көр сүреп ештіңге көрген мұманды жоқ болады. Бірау - ол дау өзер ейліп, еск сен де болып түрре мақтық, көмінбесін кіміз жақсы адамның көнісіне  жоқ. Кей бай, өзі аныратұғаны екен. Созда қылмаса, екеуді ішім келеді: «ақыр құдайғы жазарып, жауысына ұқсырып, ақыл үйрену еліп, қал жәрек болып кетсе, жамын жаққа тарамасан еке де, онайында бір күні бағып ке мақтан дап, кеңбей қалмын деп қалап жерсе, керістік жеп, асып алтанға да қылы болай

In [15]:
print(sample(rnn, 1000, prime='Е', top_k=5))

Енді ой сазлақтар - хакімерсе, едес болмаса, бұзақ балақын баласынамын. Егер беріктерінің жақсылық биретіп жерге де қарамаатып жаратып оллады деп аламыз бе, бірдің қазақ білінді білмейді, өзін- жар келіп тапса, жана қайшылыкқан соң амақ шайда болады. Монда аябып аутықтан бағқта кіргенде, бірін айтып бұр тағы болмас үй, ғылым біреуге жарамылға ұртық, иманды толық кісі болапыз. Әкбәр ман болады далғарлық, көңілге сизатының ішінде не қаламын деп, жұртыс болады. Бірлік қартық алмайтыбы бара жақылыптқа тұрмастың екені деге мер көтеді. Тагаршаны да жарсып, жарлысын, қуат жүріп, бұрық қызақ, жақ - құзыққа алмал жасып жүрек. Жарып малазының ақылығы, күнін үйренбектің тілін жес көріп, ол сөзіп жарарсыздар бірлік қылады. Дүниеде қушақ болып, тамыным жоқ, седей ме? Өздере терсе керек, сондайың біліп, үйреніп кетесіз. Бұлардың айты жере де жеріп, сол мен бірдің болмаса да болмаса, біз мәле батырмағын жауған құра, жүрік қылап таралады. Жоқдағы таланып адам да саласады. Осы көнің әмелші түрлі нәрсел

In [16]:
print(sample(rnn, 1000, prime='М', top_k=5))

Мын, яғни айтқаныңнан бары жоқ. Бірақ ойнан қалып кісіпіп жасып кетпейді екен, сатық көрмейуін деп, бірін жықсыл қай жай, кіші біреуге жасқалын көргенге, сен болады екен. Қазақ баласының баласы үшін ембекін де арып, жырек таға ма? Олай болғанда, адам өз халақта тағыладың адамдығы жақғанымды білмей таға, қарым тоқ жасса махаббат - барыққы мағалусын да пайдасын жәре туре бір жар болуға жоса малдар  әзерілік болып, асылардың бері толған. Және еу оғанға берген денерекі тур-бірдің мұным өмерлік бәрі--а ихсай, әлеутерді, бұл қазағына бар бірмей-тұрсын, тарқыста ғаларын білмек, кімді бұр болыд киммет жоқ. Сон жасқан қарындардың қатқының иманып імен керек: ол - жүқ екені. Құдай тағала жаланыша жасап қала алмасың, азылаттың ғолымдын ешірсе, жоқыны калмай де, сондай білгенге қараның жерсе екен. Қазақ ның сеніз ұялған біліміз ғылымды кеметтік. Егерде біреуін да көңілің өзі де сапатынақ берсе, мұнша шарқалық, қайда басқа, біреудің кісі ісі десең де, осы көнем мақ жатып, сайда де сөздерікпен барана

In [17]:
print(sample(rnn, 1000, prime='Қ', top_k=5))

Қайтып ұялған дәнелік енсез болған, хайуан, малынан орынға қойықтанға жүр көзе жем, сезің тілелге, келеріп жоқ. Бір жүнек ісіп, қымнен таққасым. «Яныны , ағайынып жарылып, жарым жатырсыз болған мықтығын асарап, білеудің бірін қызып жүреді. Мұндай адам баласына пайдасы келе ұятады, кімге көрік болады. Мұндай ұят қылып тапатына жалапындып, еліншен жақсы қыламыз де бөлем ғана еліп тұрағанда. Адамның аламыз ғалалы қарамасыз де селгендігіңе біреуін бахыс көремек. Олар болмаса, ләлси адам басабар. Сар бағалсың женге мен соу кеп өзгенікпек көріп жүр. Бұлар - дән болған. Бұлардың мүлкінің ибарының жеткені жөм болса, ол ниетің ғылым болса, халақ бәрлән бұзатар фиғалақұран ақылды жақсы, жарымға жұрып жаратылға тұрғанның адам баласы ғана білгенімдікпенен тайсалмақ - өнірлік иелі «дарлыр, құдай тағаланың көңілге сипатына жерсез кісілерінің бәрі - пайдасан көрін, астыңның көрсе, қайты жақсы көріп, өзін ойлап қалары еді. Мандый өсіп, өзін өлсектерсің, кірсіз білеуге жаратып, жалған без келгенім өмер

In [18]:
print(sample(rnn, 1000, prime='Ы', top_k=5))

Ырса, сендігі есі менен қанып жергені қайтып алады. Ағайын қылып, біреуін тауып құрар атып, монын асыралып, түген де жақсы ерсі де, өзің адам баласы біресұғын тәрілгені керек болады. Женге жоқ рол соң, жарлысын жоқ болады, оның жұз қалайшаған махшар ісі емес, һәмманы да барады да шығада. Оның үшін адам баласы ғылым таққаным болмаса, малды сұрап сабыбық, бір тәрті мұрсат болса екен деп, үлгенің - моның игерін солдару кісіне қорықып, сұрасын дайдып келеді. Осы болады де алланың жары болған соң, жәнн ниңнің бетіндігіна нә сәрімлін ешеп тара баладын болса, жән- иіс болмаса, бірмененің қойы баласын деп жұр боларына орынсызын, бір иманы бар келікпен болып, оны күнме?- Я түйті, аның бәрі - жәрекі деген кісірер едіс көргенде жоздығытыны көрген кісіемен соңалы осы жүнде біріндер бір өзіненіңде өзің қызғанына қашжынып бола а ма? Біреуді қарқатқа қарсыт, аруызды табады, сайлауға жарамайды, азғыртық деп арыз дағылады. Ол маланда қаліңдер месіңде көппен көрсен, қулып-түгін, қымсат, ауылс- қуанын ба